
# Marketing Campaigns — EDA & Hypothesis Testing

This notebook explores the **Marketing Mix (5 Ps)** in the provided dataset: **People, Product, Place, Promotion,** (and we treat **Price** implicitly via spending). We will clean the data, engineer features, visualize patterns, and test hypotheses about customer acquisition and channel preferences.

**Data files used:**
- `marketing_data.csv` (customer-level features & outcomes)
- `Data Dictionary - Response to marketing campaigns.xlsx` (column descriptions)

In [1]:

# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Plot settings
plt.style.use('seaborn-v0_8')
sns.set(color_codes=True)
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)


In [ ]:

# Load raw data
raw_path = 'marketing_data.csv'
df = pd.read_csv(raw_path)
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")
df.head()


In [ ]:

# 5.1 Clean Income: remove currency formatting and convert to float
if ' Income ' in df.columns:
    df.rename(columns={' Income ': 'Income'}, inplace=True)

# Remove $ and commas, strip spaces; convert missing to NaN
def _to_float(x):
    if pd.isna(x):
        return np.nan
    if isinstance(x, (int, float)):
        return float(x)
    x = str(x).strip().replace('$', '').replace(',', '')
    try:
        return float(x)
    except:
        return np.nan

if 'Income' in df.columns:
    df['Income'] = df['Income'].apply(_to_float)

# 5.2 Standardize categorical values in Education & Marital_Status
# Normalize whitespace & case
for col in ['Education', 'Marital_Status', 'Country']:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()

# Fix rare/erroneous categories
edu_map = {
    'Basic': 'Basic',
    '2n Cycle': '2n Cycle',
    'Graduation': 'Graduation',
    'Master': 'Master',
    'PhD': 'PhD'
}
# Map anything else to closest valid (fallback to original where already valid)
df['Education'] = df['Education'].replace({'YOLO': 'Graduation', 'Absurd': 'Graduation'})
df.loc[~df['Education'].isin(edu_map.keys()), 'Education'] = df.loc[~df['Education'].isin(edu_map.keys()), 'Education'].apply(lambda x: 'Graduation' if x not in edu_map else x)

marital_map = {
    'Single': 'Single',
    'Married': 'Married',
    'Together': 'Together',
    'Divorced': 'Divorced',
    'Widow': 'Widow'
}
df['Marital_Status'] = df['Marital_Status'].replace({'Alone': 'Single', 'YOLO': 'Single'})
df.loc[~df['Marital_Status'].isin(marital_map.keys()), 'Marital_Status'] = 'Single'

# 5.3 Parse Dt_Customer and engineer Age, Children, Total Spend, Total Purchases

if 'Dt_Customer' in df.columns:
    df['Dt_Customer'] = pd.to_datetime(
        df['Dt_Customer'].astype(str).str.strip(),
        format='%m/%d/%y',
        errors='coerce'
    )

# Age: use year at enrollment if available else assume 2014 (dataset horizon)
reference_year = 2014
if 'Year_Birth' in df.columns:
    df['Age'] = np.where(df['Dt_Customer'].notna(), df['Dt_Customer'].dt.year - df['Year_Birth'], reference_year - df['Year_Birth'])

# Children total
df['Children'] = df[['Kidhome', 'Teenhome']].sum(axis=1)

# Total spend across products
spend_cols = ['MntWines','MntFruits','MntMeatProducts','MntFishProducts','MntSweetProducts','MntGoldProds']
spend_cols = [c for c in spend_cols if c in df.columns]
df['Total_Spend'] = df[spend_cols].sum(axis=1)

# Total purchases across channels
purchase_cols = ['NumWebPurchases','NumCatalogPurchases','NumStorePurchases']
purchase_cols = [c for c in purchase_cols if c in df.columns]
df['Total_Purchases'] = df[purchase_cols].sum(axis=1)

# 5.4 Handle outliers: cap (winsorize) Age, Income, Total_Spend at 1st/99th percentiles
for c in ['Age','Income','Total_Spend']:
    if c in df.columns:
        lower, upper = df[c].quantile([0.01, 0.99])
        df[c] = df[c].clip(lower, upper)

# 5.5 Impute missing Income grouped by Education & Marital_Status (median within group)
if 'Income' in df.columns:
    df['Income'] = df.groupby(['Education','Marital_Status'])['Income'].transform(lambda s: s.fillna(s.median()))

# 5.6 Ordinal encode Education
education_order = ['Basic','2n Cycle','Graduation','Master','PhD']
df['Education_Ord'] = pd.Categorical(df['Education'], categories=education_order, ordered=True).codes

# Preview cleaned data
df[['ID','Year_Birth','Age','Education','Education_Ord','Marital_Status','Income','Children','Total_Spend','Total_Purchases']].head()


In [ ]:

# --- EDA Visualizations ---
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

sns.histplot(df['Age'], kde=True, ax=axes[0,0], color='steelblue')
axes[0,0].set_title('Age Distribution')

sns.histplot(df['Income'], kde=True, ax=axes[0,1], color='darkgreen')
axes[0,1].set_title('Income Distribution')

sns.histplot(df['Total_Spend'], kde=True, ax=axes[1,0], color='purple')
axes[1,0].set_title('Total Spend Distribution')

sns.histplot(df['Total_Purchases'], kde=True, ax=axes[1,1], color='orange')
axes[1,1].set_title('Total Purchases Distribution')

plt.tight_layout()

# Boxplot: Income by Education
plt.figure(figsize=(10,5))
sns.boxplot(x='Education', y='Income', data=df, order=['Basic','2n Cycle','Graduation','Master','PhD'])
plt.title('Income by Education Level')
plt.xticks(rotation=0)
plt.show()

# Correlation heatmap
numeric_cols = df.select_dtypes(include=[np.number]).columns
plt.figure(figsize=(12,9))
sns.heatmap(df[numeric_cols].corr(), cmap='coolwarm', center=0)
plt.title('Correlation Heatmap (Numeric Features)')
plt.show()


In [ ]:

# --- Hypothesis Testing ---
results = {}

# H1: Older individuals prefer in-store shopping (vs online)
# Define older as Age >= median
age_median = df['Age'].median()
df['AgeGroup'] = np.where(df['Age'] >= age_median, 'Older', 'Younger')

older_store = df.loc[df['AgeGroup']=='Older', 'NumStorePurchases']
younger_store = df.loc[df['AgeGroup']=='Younger', 'NumStorePurchases']

older_web = df.loc[df['AgeGroup']=='Older', 'NumWebPurchases']
younger_web = df.loc[df['AgeGroup']=='Younger', 'NumWebPurchases']

# t-tests
t_store = stats.ttest_ind(older_store.dropna(), younger_store.dropna(), equal_var=False)
t_web = stats.ttest_ind(older_web.dropna(), younger_web.dropna(), equal_var=False)

results['H1_store_pref'] = {'t_stat': t_store.statistic, 'p_value': t_store.pvalue}
results['H1_web_pref_inverse'] = {'t_stat': t_web.statistic, 'p_value': t_web.pvalue}

# H2: Customers with children shop more online
with_children = df.loc[df['Children']>0, 'NumWebPurchases']
no_children = df.loc[df['Children']==0, 'NumWebPurchases']
t_children = stats.ttest_ind(with_children.dropna(), no_children.dropna(), equal_var=False)
results['H2_children_online'] = {'t_stat': t_children.statistic, 'p_value': t_children.pvalue}

# H3: Cannibalization: store purchases negatively correlate with online+catalog
omni = df[['NumStorePurchases','NumWebPurchases','NumCatalogPurchases']].dropna()
omni['NonStore'] = omni['NumWebPurchases'] + omni['NumCatalogPurchases']
rho, pval = stats.pearsonr(omni['NumStorePurchases'], omni['NonStore'])
results['H3_cannibalization_corr'] = {'pearson_r': rho, 'p_value': pval}

# H4: US vs Rest in Total Purchases
if 'Country' in df.columns:
    us = df.loc[df['Country']=='US', 'Total_Purchases']
    rest = df.loc[df['Country']!='US', 'Total_Purchases']
    t_us = stats.ttest_ind(us.dropna(), rest.dropna(), equal_var=False)
    results['H4_US_vs_Rest_total_purchases'] = {'t_stat': t_us.statistic, 'p_value': t_us.pvalue,
                                               'US_mean': float(us.mean()), 'Rest_mean': float(rest.mean()),
                                               'US_n': int(us.count()), 'Rest_n': int(rest.count())}

# Show results
import pprint
pprint.pprint(results)


In [ ]:

# --- Business Insights & Visuals ---

# Top-performing products by total revenue
product_totals = df[spend_cols].sum().sort_values(ascending=False)
plt.figure(figsize=(9,5))


product_df = product_totals.reset_index()
product_df.columns = ['Product', 'Total']

sns.barplot(
    data=product_df,
    x='Product', y='Total',
    hue='Product',                 # assign hue
    palette='viridis',
    legend=False                   # same visual effect, no legend
)

plt.title('Total Revenue by Product Category (2 years)')
plt.ylabel('Total Amount')
plt.xlabel('Product')
plt.xticks(rotation=35)
plt.show()

# Acceptance rate of last campaign vs age bands
df['AgeBand'] = pd.cut(df['Age'], bins=[0,30,40,50,60,100], labels=['<30','30-39','40-49','50-59','60+'])
acc_rate_by_age = df.groupby('AgeBand', observed=True)['Response'].mean()
plt.figure(figsize=(8,4))
sns.barplot(x=acc_rate_by_age.index.astype(str), y=acc_rate_by_age.values, color='teal')
plt.title('Last Campaign Acceptance Rate by Age Band')
plt.ylabel('Acceptance Rate')
plt.xlabel('Age Band')
plt.show()

# Country with highest number of customers who accepted last campaign
acc_by_country = df.loc[df['Response']==1].groupby('Country')['ID'].count().sort_values(ascending=False)
plt.figure(figsize=(10,5))
sns.barplot(x=acc_by_country.index, y=acc_by_country.values, color='slateblue')
plt.title('Accepted Last Campaign — Customers by Country')
plt.ylabel('Count of Customers')
plt.xlabel('Country')
plt.xticks(rotation=45)
plt.show()

# Pattern: children vs total expenditure
plt.figure(figsize=(8,4))
sns.boxplot(x='Children', y='Total_Spend', data=df)
plt.title('Total Spend vs Number of Children at Home')
plt.xlabel('Children at Home')
plt.ylabel('Total Spend (2 years)')
plt.show()

# Education of customers who complained in last 2 years
complaints = df.loc[df['Complain']==1]
complaints_edu = complaints['Education'].value_counts()
plt.figure(figsize=(8,4))
sns.barplot(x=complaints_edu.index, y=complaints_edu.values, color='crimson')
plt.title('Complaints by Education Level (last 2 years)')
plt.xlabel('Education Level')
plt.ylabel('Count of Complaints')
plt.show()


In [7]:

# --- Encoding Examples ---

# Ordinal encoding already applied for Education (Education_Ord)

# One-hot for Marital_Status & Country (examples)
cat_cols = []
for c in ['Marital_Status','Country']:
    if c in df.columns:
        cat_cols.append(c)

if cat_cols:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    ohe_df = pd.DataFrame(ohe.fit_transform(df[cat_cols]))
    ohe_df.columns = ohe.get_feature_names_out(cat_cols)
    encoded_df = pd.concat([df.drop(columns=cat_cols), ohe_df], axis=1)
    encoded_df.head()
else:
    print("No categorical columns available for one-hot encoding demo.")


In [ ]:

# Save cleaned dataset for downstream work
out_path = 'marketing_data_cleaned.csv'
df.to_csv(out_path, index=False)
print(f"Cleaned dataset written to {out_path}")
